In [ ]:
!pip install spectrum
import numpy as np
import dill  # pkl eval
import random
from spectrum import *
from scipy.fftpack import fft
from scipy.stats.distributions import chi2
from scipy.stats import t
import time
from datetime import datetime
from numba import jit

old_settings = np.seterr(over='ignore', divide='ignore', invalid='ignore')

def creatdbs():
    iD = amplitude
    pulse = iD * np.ones((1, int(PW / dt)))

    i = 0
    while i < t.size:
        Idbs[int(i):(int(i + PW / dt))] = pulse
        instfreq = pattern
        isi = 1000. / instfreq
        i = i + np.round(1. / dt * isi)

    return Idbs


# Run CTX-BG-TH Network Model
def CTX_BG_TH_network():
    # initial conditions (IC) to the different cell types
    if testing_network:
        v1 = np.arange(-10., -20., -1)
        v2 = np.arange(-20., -30., -1)
        v3 = np.arange(-30., -40., -1)
        v4 = np.arange(-40., -50., -1)
        v5 = np.arange(-50., -60., -1)
        v6 = np.arange(-60., -70., -1)
    else:
        np.random.seed(10)  # fix the seed so we get the same random number at each trial
        v1 = np.random.normal(-62, 5, n)  # normal distribution with mean = -62/-63.8 and SD = 5
        v2 = np.random.normal(-62, 5, n)
        v3 = np.random.normal(-62, 5, n)
        v4 = np.random.normal(-62, 5, n)
        v5 = np.random.normal(-63.8, 5, n)
        v6 = np.random.normal(-63.8, 5, n)

    # IC-Excitatory Regular Spiking Neurons
    ae = 0.02
    be = 0.2
    ce = -65
    de = 8
    # IC-Inhibitory Fast Spiking InterNeurons
    ai = 0.1
    bi = 0.2
    ci = -65
    di = 2

    Cm = 1  # Membrane capacitance

    # Ionic conductance and Equilibrium potential values

    gl = np.array([0.05, 0.35, 0.1, 0.1])
    El = np.array([-70, -60, -65, -67])
    gna = np.array([3, 49, 120, 100])
    Ena = np.array([50, 60, 55, 50])
    gk = np.array([5, 57, 30, 80])
    Ek = np.array([-75, -90, -80, -100])
    gt = np.array([5, 5, 0.5])
    Et = 0
    gca = np.array([0, 2, 0.15])
    Eca = np.array([0, 140, 120])
    Em = -100
    gahp = np.array([0, 20, 10])
    k1 = np.array([0, 15, 10])
    kca = np.array([0, 22.5, 15])
    ga = 5
    gL = 15
    gcak = 1
    Kca = 2 * 10 ** (-3)
    Z = 2
    F = 96485
    CAsn2 = 0.005 * np.ones(n)
    Cao = 2000
    R = 8314
    T = 298
    alp = 1 / (Z * F)
    con = (R * T) / (Z * F)

    # Setting initial matrices
    vth = np.zeros([n, ntot])  # thalamic membrane voltage
    vsn = np.zeros([n, ntot])  # STN membrane voltage
    vge = np.zeros([n, ntot])  # GPe membrane voltage
    vgi = np.zeros([n, ntot])  # GPi membrane voltage
    vstr_indr = np.zeros([n, ntot])  # Indirect Striatum membrane voltage
    vstr_dr = np.zeros([n, ntot])  # Direct Striatum membrane voltage
    ve = np.zeros([n, ntot])  # Excitatory Cortex membrane voltage
    vi = np.zeros([n, ntot])  # Inhibitory Cortex membrane voltage
    ue = np.zeros([n, ntot])
    ui = np.zeros([n, ntot])

    ##initial conditions
    vth[:, 0] = v1
    vsn[:, 0] = v2
    vge[:, 0] = v3
    vgi[:, 0] = v4
    vstr_indr[:, 0] = v5
    vstr_dr[:, 0] = v6
    ve[:, 0] = ce
    ue[:, 0] = be * ve[:, 0]
    vi[:, 0] = ci
    ui[:, 0] = bi * vi[:, 0]

    ## State variables
    N3 = gpe_ninf(vge[:, 0])
    N4 = gpe_ninf(vgi[:, 0])
    H1 = th_hinf(vth[:, 0])
    H3 = gpe_hinf(vge[:, 0])
    H4 = gpe_hinf(vgi[:, 0])
    R1 = th_rinf(vth[:, 0])
    R3 = gpe_rinf(vge[:, 0])
    R4 = gpe_rinf(vgi[:, 0])
    CA3 = CA4 = 0.1

    N2 = stn_ninf(vsn[:, 0])
    H2 = stn_hinf(vsn[:, 0])
    M2 = stn_minf(vsn[:, 0])
    A2 = stn_ainf(vsn[:, 0])
    B2 = stn_binf(vsn[:, 0])
    C2 = stn_cinf(vsn[:, 0])
    D2 = stn_d2inf(vsn[:, 0])
    D1 = stn_d1inf(vsn[:, 0])
    P2 = stn_pinf(vsn[:, 0])
    Q2 = stn_qinf(vsn[:, 0])
    R2 = stn_rinf(vsn[:, 0])

    m5 = alpham(vstr_indr[:, 0]) / (alpham(vstr_indr[:, 0]) + betam(vstr_indr[:, 0]))
    h5 = alphah(vstr_indr[:, 0]) / (alphah(vstr_indr[:, 0]) + betah(vstr_indr[:, 0]))
    n5 = alphan(vstr_indr[:, 0]) / (alphan(vstr_indr[:, 0]) + betan(vstr_indr[:, 0]))
    p5 = alphap(vstr_indr[:, 0]) / (alphap(vstr_indr[:, 0]) + betap(vstr_indr[:, 0]))
    m6 = alpham(vstr_dr[:, 0]) / (alpham(vstr_dr[:, 0]) + betam(vstr_dr[:, 0]))
    h6 = alphah(vstr_dr[:, 0]) / (alphah(vstr_dr[:, 0]) + betah(vstr_dr[:, 0]))
    n6 = alphan(vstr_dr[:, 0]) / (alphan(vstr_dr[:, 0]) + betan(vstr_dr[:, 0]))
    p6 = alphap(vstr_dr[:, 0]) / (alphap(vstr_dr[:, 0]) + betap(vstr_dr[:, 0]))

    # Synapse parameters
    Esyn = np.array([-85, 0, -85, 0, -85, -85, -80])
    tau = 5
    tau_i = 13
    gpeak = 0.43
    gpeak1 = 0.3

    S2a = S21a = S2b = S21b = S2an = S21an = np.zeros(n)
    S3a = S31a = S3b = S31b = S32b = S3c = S31c = S32c = np.zeros(n)
    S4 = np.zeros(n)
    S5 = S51 = S52 = S53 = S54 = S55 = S56 = S57 = S58 = S59 = np.zeros(n)
    S6a = S6b = S6bn = S61bn = S61b = np.zeros(n)
    S9 = S91 = S92 = S93 = S94 = S95 = S96 = S97 = S98 = S99 = np.zeros(n)
    S7 = np.zeros(n)
    S8 = np.zeros(n)
    S1a = S1b = S1c = np.zeros(n)
    Z1a = Z1b = np.zeros(n)

    t_a = 1000  # Max duration of syn conductance
    ntot1 = int(t_a / dt + 1)  # synapse counts
    tr3 = 30
    tr4 = 30
    td2 = 130
    tr2 = 2
    t_vec = np.linspace(0, t_a, ntot1)

    const = gpeak / (tau * np.exp(-1))
    const1 = gpeak1 / (tau * np.exp(-1))
    const2 = gpeak1 / (tau * np.exp(-1))

    # TH-CTX Synapse
    t_d_th_cor = 5
    syn_func_th = const * (t_vec - t_d_th_cor)  # *(np.exp(-(t_vec-t_d_th_cor)/tau))*((t_vec>=t_d_th_cor)&(t_vec<=t_a))

    # %%

    # STN-GPe Synapse
    t_d_stn_gpe = 2
    taudstngpea = 2.5
    taurstngpea = 0.4
    taudstngpen = 67
    taurstngpen = 2
    # tpeakstngpea = 2.87266 # this is obtained from the next line, I just put it there to minmize the calculations
    tpeakstngpea = t_d_stn_gpe + (
            ((taudstngpea * taurstngpea) / (taudstngpea - taurstngpea)) * np.log(taudstngpea / taurstngpea))
    # fstngpea     = 1.68778
    fstngpea = 1 / (np.exp(-(tpeakstngpea - t_d_stn_gpe) / taudstngpea) - np.exp(-(tpeakstngpea - t_d_stn_gpe) / taurstngpea))
    syn_func_stn_gpea = gpeak * fstngpea * (
            np.exp(-(t_vec - t_d_stn_gpe) / taudstngpea) - np.exp(-(t_vec - t_d_stn_gpe) / taurstngpea)) * (
                                    (t_vec >= t_d_stn_gpe) & (t_vec <= t_a))
    # tpeakstngpen = 9.23919
    temp2 = np.log(taudstngpen / taurstngpen)
    tpeakstngpen = t_d_stn_gpe + (((taudstngpen * taurstngpen) / (taudstngpen - taurstngpen)) * temp2)
    # fstngpen     = 1.14838
    fstngpen = 1 / (np.exp(-(tpeakstngpen - t_d_stn_gpe) / taudstngpen) - np.exp(-(tpeakstngpen - t_d_stn_gpe) / taurstngpen))
    syn_func_stn_gpen = gpeak * fstngpen * ((t_vec >= t_d_stn_gpe) & (t_vec <= t_a)) * (
            np.exp(-(t_vec - t_d_stn_gpe) / taudstngpen) - np.exp(-(t_vec - t_d_stn_gpe) / taurstngpen))

    # STN-GPi Synapse
    t_d_stn_gpi = 1.5
    syn_func_stn_gpi = const * (t_vec - t_d_stn_gpi) * (
            (t_vec >= t_d_stn_gpi) & (t_vec <= t_a))  *(np.exp(-(t_vec-t_d_stn_gpi)/tau))

    # GPe-STN Synapse
    t_d_gpe_stn = 4
    taudg = 7.7
    taurg = 0.4
    # tpeakg = 5.24783
    # fg = 1.24036
    tempp = np.log(taudg / taurg)
    tpeakg = t_d_gpe_stn + (((taudg * taurg) / (taudg - taurg)) * tempp)
    fg = 1 / (np.exp(-(tpeakg - t_d_gpe_stn) / taudg) - np.exp(-(tpeakg - t_d_gpe_stn) / taurg))
    syn_func_gpe_stn = gpeak1 * fg * ((t_vec >= t_d_gpe_stn) & (
            t_vec <= t_a))  *(np.exp(-(t_vec-t_d_gpe_stn)/taudg)-np.exp(-(t_vec-t_d_gpe_stn)/taurg))

    # GPe-GPi Synapse
    t_d_gpe_gpi = 3
    syn_func_gpe_gpi = const1 * (
            t_vec - t_d_gpe_gpi)  *(np.exp(-(t_vec-t_d_gpe_gpi)/tau))*((t_vec>=t_d_gpe_gpi)&(t_vec<=t_a))

    # GPe-GPe Synapse
    t_d_gpe_gpe = 1
    syn_func_gpe_gpe = const1 * (
            t_vec - t_d_gpe_gpe)  *(np.exp(-(t_vec-t_d_gpe_gpe)/tau))*((t_vec>=t_d_gpe_gpe)&(t_vec<=t_a))

    # GPi-TH Synapse
    t_d_gpi_th = 5
    syn_func_gpi_th = const1 * (
                t_vec - t_d_gpi_th)  *(np.exp(-(t_vec-t_d_gpi_th)/tau))*((t_vec>=t_d_gpi_th)&(t_vec<=t_a))

    # Indirect Str-GPe Synapse
    t_d_d2_gpe = 5
    syn_func_str_indr = const2 * (
            t_vec - t_d_d2_gpe)  *(np.exp(-(t_vec-t_d_d2_gpe)/tau))*((t_vec>=t_d_d2_gpe)&(t_vec<=t_a))

    # direct Str-GPi Synapse
    t_d_d1_gpi = 4
    syn_func_str_dr = const2 * (
                t_vec - t_d_d1_gpi)  *(np.exp(-(t_vec-t_d_d1_gpi)/tau))*((t_vec>=t_d_d1_gpi)&(t_vec<=t_a))

    # Cortex-Indirect Str Synapse
    t_d_cor_d2 = 5.1
    syn_func_cor_d2 = const * (t_vec - t_d_cor_d2)  *(np.exp(-(t_vec-t_d_cor_d2)/tau))*((t_vec>=t_d_cor_d2)&(t_vec<=t_a))

    # Cortex-STN Synapse
    t_d_cor_stn = 5.9
    taudn = 90
    taurn = 2
    tauda = 2.49
    taura = 0.5
    temp = np.log(tauda / taura)
    tpeaka = t_d_cor_stn + (((tauda * taura) / (tauda - taura)) * temp)
    fa = 1 / (np.exp(-(tpeaka - t_d_cor_stn) / tauda) - np.exp(-(tpeaka - t_d_cor_stn) / taura))
    syn_func_cor_stn_a = gpeak * fa * ((t_vec >= t_d_cor_stn) & (
                t_vec <= t_a))  *(np.exp(-(t_vec-t_d_cor_stn)/tauda)-np.exp(-(t_vec-t_d_cor_stn)/taura))
    temp1 = np.log(taudn / taurn)
    tpeakn = t_d_cor_stn + (((taudn * taurn) / (taudn - taurn)) * temp1)
    fn = 1 / (np.exp(-(tpeakn - t_d_cor_stn) / taudn) - np.exp(-(tpeakn - t_d_cor_stn) / taurn))
    syn_func_cor_stn_n = gpeak * fn * ((t_vec >= t_d_cor_stn) & (t_vec <= t_a)) * (
                np.exp(-(t_vec - t_d_cor_stn) / taudn) - np.exp(-(t_vec - t_d_cor_stn) / taurn))

    # %%


    t_list_th = np.empty((n,), dtype=object)  # intead of strcut I am just using arrays!
    t_list_th.fill(np.array([], dtype=int))
    t_list_cor = np.empty((n,), dtype=object)
    t_list_cor.fill(np.array([], dtype=int))
    t_list_str_indr = np.empty((n,), dtype=object)
    t_list_str_indr.fill(np.array([], dtype=int))
    t_list_str_dr = np.empty((n,), dtype=object)
    t_list_str_dr.fill(np.array([], dtype=int))
    t_list_stn = np.empty((n,), dtype=object)
    t_list_stn.fill(np.array([], dtype=int))
    t_list_gpe = np.empty((n,), dtype=object)
    t_list_gpe.fill(np.array([], dtype=int))
    t_list_gpi = np.empty((n,), dtype=object)
    t_list_gpi.fill(np.array([], dtype=int))

    if testing_network:
        alll = np.arange(0, 10, 1)
        bll = np.arange(0, 10, 1)
        cll = np.arange(0, 10, 1)
        dll = np.arange(0, 10, 1)
        ell = np.arange(0, 10, 1)
        fll = np.arange(0, 10, 1)
        gll = np.arange(0, 10, 1)
        hll = np.arange(0, 10, 1)
        ill = np.arange(0, 10, 1)
        jll = np.arange(0, 10, 1)
        kll = np.arange(0, 10, 1)
        lll = np.arange(0, 10, 1)
        mll = np.arange(0, 10, 1)
        nll = np.arange(0, 10, 1)
        oll = np.arange(0, 10, 1)

        fixrand = 0.5 * np.ones(n)
        gcorsna = 0.3 * fixrand
        gcorsnn = 0.003 * fixrand
        gcordrstr = (0.07 - 0.044 * pd) + 0.001 * fixrand
        ggege = fixrand
        gsngen = np.zeros(n)
        gsngea = np.zeros(n)
        gsngi = np.zeros(n)
        randperm = np.arange(0, 10, 1)
        gsngen[randperm[0]] = 0.002 * 0.25
        gsngen[randperm[1]] = 0.002 * 0.75

        gsngea[randperm[0]] = 0.3 * 0.25
        gsngea[randperm[1]] = 0.3 * 0.75

        gsngi[randperm[0:5]] = 0.15
    else:
        alll = random.sample(range(n), n)
        bll = random.sample(range(n), n)
        cll = random.sample(range(n), n)
        dll = random.sample(range(n), n)
        ell = random.sample(range(n), n)
        fll = random.sample(range(n), n)
        gll = random.sample(range(n), n)
        hll = random.sample(range(n), n)
        ill = random.sample(range(n), n)
        jll = random.sample(range(n), n)
        kll = random.sample(range(n), n)
        lll = random.sample(range(n), n)
        mll = random.sample(range(n), n)
        nll = random.sample(range(n), n)
        oll = random.sample(range(n), n)

        gcorsna = 0.3 * np.random.rand(n)
        gcorsnn = 0.003 * np.random.rand(n)
        gcordrstr = (0.07 - 0.044 * pd) + 0.001 * np.random.rand(n)
        ggege = np.random.rand(n)
        gsngen = np.zeros(n)
        gsngea = np.zeros(n)
        gsngi = np.zeros(n)
        randperm = np.random.permutation(n)
        gsngen[randperm[0]] = 0.002 * np.random.rand(1)
        gsngen[randperm[1]] = 0.002 * np.random.rand(1)

        randperm = np.random.permutation(n)
        gsngea[randperm[0]] = 0.3 * np.random.rand(1)
        gsngea[randperm[1]] = 0.3 * np.random.rand(1)

        randperm = np.random.permutation(n)
        gsngi[randperm[0:5]] = 0.15

    ggith = 0.112
    ggesn = 0.5
    gstrgpe = 0.5
    gstrgpi = 0.5
    ggigi = 0.5
    gm = 1
    ggaba = 0.1
    gcorindrstr = 0.07
    gie = 0.2
    gthcor = 0.15
    gei = 0.1

    for i in range(1, len(t)):  # change to ntot


        #start = time.time()
        # print('i = ', i)

        V1 = vth[:, i - 1]
        V2 = vsn[:, i - 1]
        V3 = vge[:, i - 1]
        V4 = vgi[:, i - 1]
        V5 = vstr_indr[:, i - 1]
        V6 = vstr_dr[:, i - 1]
        V7 = ve[:, i - 1]
        V8 = vi[:, i - 1]

        S21a[1:n] = S2a[0:n - 1]
        S21a[0] = S2a[n - 1]
        S21an[1:n] = S2an[0:n - 1]
        S21an[0] = S2an[n - 1]
        S21b[1:n] = S2b[0:n - 1]
        S21b[0] = S2b[n - 1]
        S31a[0:n - 1] = S3a[1:n]
        S31a[n - 1] = S3a[0]
        S31b[0:n - 1] = S3b[1:n]
        S31b[n - 1] = S3b[0]
        S31c[0:n - 1] = S3c[1:n]
        S31c[n - 1] = S3c[0]
        S32c[2:n] = S3c[0:n - 2]
        S32c[0:2] = S3c[n - 2:n]
        S32b[2:n] = S3b[0:n - 2]
        S32b[0:2] = S3b[n - 2:n]
        S11cr = S1c[alll]
        S12cr = S1c[bll]
        S13cr = S1c[cll]
        S14cr = S1c[dll]
        S11br = S1b[ell]
        S12br = S1b[fll]
        S13br = S1b[gll]
        S14br = S1b[hll]
        S11ar = S1a[ill]
        S12ar = S1a[jll]
        S13ar = S1a[kll]
        S14ar = S1a[lll]
        S81r = S8[mll]
        S82r = S8[nll]
        S83r = S8[oll]
        S51[0:n - 1] = S5[1:n]
        S51[n - 1] = S5[0]
        S52[0:n - 2] = S5[2:n]
        S52[n - 2:n] = S5[0:2]
        S53[0:n - 3] = S5[3:n]
        S53[n - 3:n] = S5[0:3]
        S54[0:n - 4] = S5[4:n]
        S54[n - 4:n] = S5[0:4]
        S55[0:n - 5] = S5[5:n]
        S55[n - 5:n] = S5[0:5]
        S56[0:n - 6] = S5[6:n]
        S56[n - 6:n] = S5[0:6]
        S57[0:n - 7] = S5[7:n]
        S57[n - 7:n] = S5[0:7]
        S58[0:n - 8] = S5[8:n]
        S58[n - 8:n] = S5[0:8]
        S59[0:n - 9] = S5[9:n]
        S59[n - 9:n] = S5[0:9]
        S61b[0:n - 1] = S6b[1:n]
        S61b[n - 1] = S6b[0]
        S61bn[0:n - 1] = S6bn[1:n]
        S61bn[n - 1] = S6bn[0]

        S91[0:n - 1] = S9[1:n]
        S91[n - 1] = S9[0]
        S92[0:n - 2] = S9[2:n]
        S92[n - 2:n] = S9[0:2]
        S93[0:n - 3] = S9[3:n]
        S93[n - 3:n] = S9[0:3]
        S94[0:n - 4] = S9[4:n]
        S94[n - 4:n] = S9[0:4]
        S95[0:n - 5] = S9[5:n]
        S95[n - 5:n] = S9[0:5]
        S96[0:n - 6] = S9[6:n]
        S96[n - 6:n] = S9[0:6]
        S97[0:n - 7] = S9[7:n]
        S97[n - 7:n] = S9[0:7]
        S98[0:n - 8] = S9[8:n]
        S98[n - 8:n] = S9[0:8]
        S99[0:n - 9] = S9[9:n]
        S99[n - 9:n] = S9[0:9]

        m1 = th_minf(V1)
        m3 = gpe_minf(V3)
        m4 = gpe_minf(V4)
        n3 = gpe_ninf(V3)
        n4 = gpe_ninf(V4)
        h1 = th_hinf(V1)
        h3 = gpe_hinf(V3)
        h4 = gpe_hinf(V4)
        p1 = th_pinf(V1)
        a3 = gpe_ainf(V3)
        a4 = gpe_ainf(V4)
        s3 = gpe_sinf(V3)
        s4 = gpe_sinf(V4)
        r1 = th_rinf(V1)
        r3 = gpe_rinf(V3)
        r4 = gpe_rinf(V4)
        tn3 = gpe_taun(V3)
        tn4 = gpe_taun(V4)
        th1 = th_tauh(V1)
        th3 = gpe_tauh(V3)
        th4 = gpe_tauh(V4)
        tr1 = th_taur(V1)

        n2 = stn_ninf(V2)
        m2 = stn_minf(V2)
        h2 = stn_hinf(V2)
        a2 = stn_ainf(V2)
        b2 = stn_binf(V2)
        c2 = stn_cinf(V2)
        d2 = stn_d2inf(V2)
        d1 = stn_d1inf(V2)
        p2 = stn_pinf(V2)
        q2 = stn_qinf(V2)
        r2 = stn_rinf(V2)

        tn2 = stn_taun(V2)
        tm2 = stn_taum(V2)
        th2 = stn_tauh(V2)
        ta2 = stn_taua(V2)
        tb2 = stn_taub(V2)
        tc2 = stn_tauc(V2)
        td1 = stn_taud1(V2)
        tp2 = stn_taup(V2)
        tq2 = stn_tauq(V2)

        Ecasn = con * np.log(Cao / CAsn2)

        # thalamic cell currents
        Il1 = gl[0] * (V1 - El[0])
        Ina1 = gna[0] * (m1 ** 3) * H1 * (V1 - Ena[0])
        Ik1 = gk[0] * ((0.75 * (1 - H1)) ** 4) * (V1 - Ek[0])
        It1 = gt[0] * (p1 ** 2) * R1 * (V1 - Et)
        Igith = ggith * (V1 - Esyn[5]) * S4
        Iappth = 1.2

        # STN cell currents
        Ina2 = gna[1] * (M2 ** 3) * H2 * (V2 - Ena[1])
        Ik2 = gk[1] * (N2 ** 4) * (V2 - Ek[1])
        Ia2 = ga * (A2 ** 2) * B2 * (V2 - Ek[1])
        IL2 = gL * (C2 ** 2) * D1 * D2 * (V2 - Ecasn)
        It2 = (gt[1] * (P2 ** 2) * Q2 * (V2 - Ecasn))
        Icak2 = gcak * (R2 ** 2) * (V2 - Ek[1])
        Il2 = gl[1] * (V2 - El[1])
        Igesn = (ggesn * ((V2 - Esyn[0]) * (S3a + S31a)))
        Icorsnampa = gcorsna * (V2 - Esyn[1]) * (S6b + S61b)
        Icorsnnmda = gcorsnn * (V2 - Esyn[1]) * (S6bn + S61bn)

        # GPe cell currents
        Il3 = gl[2] * (V3 - El[2])
        Ik3 = gk[2] * (N3 ** 4) * (V3 - Ek[2])
        Ina3 = gna[2] * (m3 ** 3) * H3 * (V3 - Ena[2])
        It3 = gt[2] * (a3 ** 3) * R3 * (V3 - Eca[2])
        Ica3 = gca[2] * (s3 ** 2) * (V3 - Eca[2])
        Iahp3 = gahp[2] * (V3 - Ek[2]) * (CA3 / (CA3 + k1[2]))
        Isngeampa = gsngea * ((V3 - Esyn[1]) * (S2a + S21a))
        Isngenmda = gsngen * ((V3 - Esyn[1]) * (S2an + S21an))
        Igege = (0.25 * (pd * 3 + 1)) * ggege * ((V3 - Esyn[2]) * (S31c + S32c))
        Istrgpe = gstrgpe * (V3 - Esyn[5]) * (S5 + S51 + S52 + S53 + S54 + S55 + S56 + S57 + S58 + S59)
        Iappgpe = 3 - 2 * corstim * (
                    1 - pd)  # Modulation only during cortical stim to maintain mean firing rate ~pd=0 if pd!=0
        # ~pd=1 if pd ==0

        # GPi cell currents
        Il4 = gl[2] * (V4 - El[2])
        Ik4 = gk[2] * (N4 ** 4) * (V4 - Ek[2])
        Ina4 = gna[2] * (m4 ** 3) * H4 * (V4 - Ena[2])
        It4 = gt[2] * (a4 ** 3) * R4 * (V4 - Eca[2])
        Ica4 = gca[2] * (s4 ** 2) * (V4 - Eca[2])
        Iahp4 = gahp[2] * (V4 - Ek[2]) * (CA4 / (CA4 + k1[2]))
        Isngi = gsngi * ((V4 - Esyn[3]) * (S2b + S21b))
        Igigi = ggigi * ((V4 - Esyn[4]) * (S31b + S32b))
        Istrgpi = gstrgpi * (V4 - Esyn[5]) * (S9 + S91 + S92 + S93 + S94 + S95 + S96 + S97 + S98 + S99)
        Iappgpi = 3

        # Striatum D2 cell currents
        Ina5 = gna[3] * (m5 ** 3) * h5 * (V5 - Ena[3])
        Ik5 = gk[3] * (n5 ** 4) * (V5 - Ek[3])
        Il5 = gl[3] * (V5 - El[3])
        Im5 = (2.6 - 1.1 * pd) * gm * p5 * (V5 - Em)
        Igaba5 = (ggaba / 4) * (V5 - Esyn[6]) * (S11cr + S12cr + S13cr + S14cr)
        Icorstr5 = gcorindrstr * (V5 - Esyn[1]) * S6a

        # Striatum D1 cell currents
        Ina6 = gna[3] * (m6 ** 3) * h6 * (V6 - Ena[3])
        Ik6 = gk[3] * (n6 ** 4) * (V6 - Ek[3])
        Il6 = gl[3] * (V6 - El[3])
        Im6 = (2.6 - 1.1 * pd) * gm * p6 * (V6 - Em)
        Igaba6 = (ggaba / 3) * (V6 - Esyn[6]) * (S81r + S82r + S83r)
        Icorstr6 = gcordrstr * (V6 - Esyn[1]) * S6a

        # Excitatory Neuron Currents
        Iie = gie * (V7 - Esyn[0]) * (S11br + S12br + S13br + S14br)
        Ithcor = gthcor * (V7 - Esyn[1]) * S7

        # Inhibitory Neuron Currents
        Iei = gei * (V8 - Esyn[1]) * (S11ar + S12ar + S13ar + S14ar)

        fvth = -Il1 - Ik1 - Ina1 - It1 - Igith + Iappth
        vth[:, i] = V1 + dt * (fvth / Cm)
        H1 = H1 + dt * ((h1 - H1) / th1)
        R1 = R1 + dt * ((r1 - R1) / tr1)

        #
        for j in range(0, n):

            if vth[j, i - 1] < -10 < vth[j, i]:
                t_list_th[j] = np.append(t_list_th[j], 1)  # add to a vector
            # Calculate synaptic current due to current and past input spikes
            if t_list_th[j].size:
                S7[j] = 0 #np.sum(syn_func_th[t_list_th[j]])
                #S7[j] = const * (np.sum(t_list_th[j])*t_a/ntot1 - t_d_th_cor*t_list_th[j].shape[0])

            # Update spike times
                t_list_th[j] += 1
                if t_list_th[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_th[j] = t_list_th[j][1:np.max(np.size(t_list_th[j]))+1]

        vsn[:, i] = V2 + dt * (1 / Cm * (
                    -Ina2 - Ik2 - Ia2 - IL2 - It2 - Icak2 - Il2 - Igesn - Icorsnampa - Icorsnnmda + Idbs[i]))  # STN-DBS

        N2 += dt * ((n2 - N2) / tn2)
        H2 += dt * ((h2 - H2) / th2)
        M2 += dt * ((m2 - M2) / tm2)
        A2 += dt * ((a2 - A2) / ta2)
        B2 += dt * ((b2 - B2) / tb2)
        C2 += dt * ((c2 - C2) / tc2)
        D2 += dt * ((d2 - D2) / td2)
        D1 += dt * ((d1 - D1) / td1)
        P2 += dt * ((p2 - P2) / tp2)
        Q2 += dt * ((q2 - Q2) / tq2)
        R2 += dt * ((r2 - R2) / tr2)

        ### Ali
        CAsn2 += dt * ((-alp * (IL2 + It2)) - (Kca * CAsn2))
        for j in range(0, n):
            if vsn[j, i - 1] < -10 < vsn[j, i]:
                t_list_stn[j] = np.append(t_list_stn[j], 1)# check for input spike

            #   Calculate synaptic current due to current and past input spikes
            if t_list_stn[j].size:
                S2a[j] = 0 #np.sum(syn_func_stn_gpea[t_list_stn[j]])
                S2an[j] = 0 #np.sum(syn_func_stn_gpen[t_list_stn[j]])
                S2b[j] = 0 #np.sum(syn_func_stn_gpi[t_list_stn[j]])

                # Update spike times
                t_list_stn[j] += 1
                if t_list_stn[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_stn[j] = t_list_stn[j][1:np.max(np.size(t_list_stn[j]))+1]

        # GPe
        vge[:, i] = V3 + dt * (1 / Cm * (
                    -Il3 - Ik3 - Ina3 - It3 - Ica3 - Iahp3 - Isngeampa - Isngenmda - Igege - Istrgpe + Iappgpe))
        N3 += dt * (0.1 * (n3 - N3) / tn3)
        H3 += dt * (0.05 * (h3 - H3) / th3)
        R3 += dt * (1 * (r3 - R3) / tr3)
        CA3 += dt * (1 * 10 ** (-4) * (-Ica3 - It3 - kca[2] * CA3))

        for j in range(0, n):

            if vge[j, i - 1] < -10 < vge[j, i]:  # check for input spike
                t_list_gpe[j] = np.append(t_list_gpe[j], 1)

            # Calculate synaptic current due to current and past input spikes
            if t_list_gpe[j].size:
                S3a[j] = 0 #np.sum(syn_func_gpe_stn[t_list_gpe[j]])
                S3b[j] = 0 #np.sum(syn_func_gpe_gpi[t_list_gpe[j]])
                S3c[j] = 0 #np.sum(syn_func_gpe_gpe[t_list_gpe[j]])

            # Update spike times

                t_list_gpe[j] += 1
                if t_list_gpe[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_gpe[j] = t_list_gpe[j][1:np.max(np.size(t_list_gpe[j]))+1]

        # GPi
        vgi[:, i] = V4 + dt * (1 / Cm * (-Il4 - Ik4 - Ina4 - It4 - Ica4 - Iahp4 - Isngi - Igigi - Istrgpi + Iappgpi))
        N4 += dt * (0.1 * (n4 - N4) / tn4)
        H4 += dt * (0.05 * (h4 - H4) / th4)
        R4 += dt * (1 * (r4 - R4) / tr4)
        CA4 += dt * (1 * 10 ** (-4) * (-Ica4 - It4 - kca[2] * CA4))

        for j in range(0, n):

            if vgi[j, i - 1] < -10 < vgi[j, i]:  # check for input spike
                t_list_gpi[j] = np.append(t_list_gpi[j], 1)

            # Calculate synaptic current due to current and past input spikes
            if t_list_gpi[j].size:
                S4[j] = 0 #np.sum(syn_func_gpi_th[t_list_gpi[j]])

            # Update spike times
                t_list_gpi[j] += 1
                if t_list_gpi[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_gpi[j] = t_list_gpi[j][1:np.max(np.size(t_list_gpi[j]))+1]

        # Striatum D2
        vstr_indr[:, i] = V5 + (dt / Cm) * (-Ina5 - Ik5 - Il5 - Im5 - Igaba5 - Icorstr5)
        m5 += dt * (alpham(V5) * (1 - m5) - betam(V5) * m5)
        h5 += dt * (alphah(V5) * (1 - h5) - betah(V5) * h5)
        n5 += dt * (alphan(V5) * (1 - n5) - betan(V5) * n5)
        p5 += dt * (alphap(V5) * (1 - p5) - betap(V5) * p5)
        S1c += dt * ((Ggaba(V5) * (1 - S1c)) - (S1c / tau_i))

        for j in range(0, n):

            if vstr_indr[j, i - 1] < -10 < vstr_indr[j, i]:  # check for input spike
                t_list_str_indr[j] = np.append(t_list_str_indr[j], 1)

            # Calculate synaptic current due to current and past input spikes
            if t_list_str_indr[j].size:
                S5[j] = 0 #np.sum(syn_func_str_indr[t_list_str_indr[j]])

            #   # Update spike times
                t_list_str_indr[j] += 1
                if t_list_str_indr[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_str_indr[j] = t_list_str_indr[j][1:np.max(np.size(t_list_str_indr[j]))+1]

        # Striatum D1 type
        vstr_dr[:, i] = V6 + (dt / Cm) * (-Ina6 - Ik6 - Il6 - Im6 - Igaba6 - Icorstr6)
        m6 += dt * (alpham(V6) * (1 - m6) - betam(V6) * m6)
        h6 += dt * (alphah(V6) * (1 - h6) - betah(V6) * h6)
        n6 += dt * (alphan(V6) * (1 - n6) - betan(V6) * n6)
        p6 += dt * (alphap(V6) * (1 - p6) - betap(V6) * p6)
        S8 += dt * ((Ggaba(V6) * (1 - S8)) - (S8 / tau_i))

        for j in range(0, n):

            if vstr_dr[j, i - 1] < -10 < vstr_dr[j, i]:  # check for input spike
                t_list_str_dr[j] = np.append(t_list_str_dr[j], 1)

            # Calculate synaptic current due to current and past input spikes
            if t_list_str_dr[j].size:
                S9[j] = np.sum(syn_func_str_dr[t_list_str_dr[j]])

            # Update spike times
                t_list_str_dr[j] += 1
                if t_list_str_dr[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_str_dr[j] = t_list_str_dr[j][1:np.max(np.size(t_list_str_dr[j]))+1]

        # Excitatory Neuron
        ve[:, i] = V7 + dt * ((0.04 * (V7 ** 2)) + (5 * V7) + 140 - ue[:, i - 1] - Iie - Ithcor + Iappco[i])
        ue[:, i] = ue[:, i - 1] + dt * (ae * ((be * V7) - ue[:, i - 1]))

        for j in range(0, n):
            if ve[j, i - 1] >= 30:
                ve[j, i] = ce
                ue[j, i] = ue[j, i - 1] + de

                t_list_cor[j] = np.append(t_list_cor[j], 1)

            # Calculate synaptic current due to current and past input spikes
            if t_list_cor[j].size:
                S6a[j] = 0 #np.sum(syn_func_cor_d2[t_list_cor[j]])
                S6b[j] = 0 #np.sum(syn_func_cor_stn_a[t_list_cor[j]])
                S6bn[j] = 0 #np.sum(syn_func_cor_stn_n[t_list_cor[j]])

            # Update spike times
                t_list_cor[j] += 1
                if t_list_cor[j][0] == t_a / dt:  # Reached max duration of syn conductance
                    t_list_cor[j] = t_list_cor[j][1:np.max(np.size(t_list_cor[j]))+1]

        ace = np.logical_and(ve[:, i - 1] < -10, ve[:, i] > -10)
        uce = np.zeros(n)
        uce[ace] = gpeak / (tau * np.exp(-1)) / dt
        S1a += dt * Z1a
        z1adot = uce - 2 / tau * Z1a - 1 / (tau ** 2) * S1a
        Z1a += dt * z1adot

        # Inhibitory InterNeuron
        vi[:, i] = V8 + dt * ((0.04 * (V8 ** 2)) + (5 * V8) + 140 - ui[:, i - 1] - Iei + Iappco[i])
        ui[:, i] = ui[:, i - 1] + dt * (ai * ((bi * V8) - ui[:, i - 1]))

        for j in range(0, n):
            if vi[j, i - 1] >= 30:
                vi[j, i] = ci
                ui[j, i] = ui[j, i - 1] + di

        aci = np.logical_and(vi[:, i - 1] < -10, vi[:, i] > -10)
        uci = np.zeros(n)
        uci[aci] = gpeak / (tau * np.exp(-1)) / dt
        S1b += dt * Z1b
        z1bdot = uci - 2 / tau * Z1b - 1 / (tau ** 2) * S1b
        Z1b += dt * z1bdot

    iTH_APs = find_spike_times(vth, t, n)
    iSTN_APs = find_spike_times(vsn, t, n)
    iGPe_APs = find_spike_times(vge, t, n)
    iGPi_APs = find_spike_times(vgi, t, n)
    iStriat_APs_indr = find_spike_times(vstr_indr, t, n)
    iStriat_APs_dr = find_spike_times(vstr_dr, t, n)
    iCor_APs = find_spike_times(np.append(ve, vi, axis=0), t, 2 * n)

    return iTH_APs, iSTN_APs, iGPe_APs, iGPi_APs, iStriat_APs_indr, iStriat_APs_dr, iCor_APs


def find_spike_times(v, it, nn):
    data = np.empty((nn,), dtype=object)
    data.fill(np.array([], dtype=int))
    it = it / 1000  # unit: second
    it = np.delete(it, -1, 0)
    for k in range(0, nn):
        data[k] = it[np.diff((v[k, :] > -20).astype(int)) > 0.5]
    return data


class paramsClass:
    dt1 = 0.01 * 10 ** (-3)
    Fs = 100000  # 1/dt
    fpass = np.array([1, 100])
    tapers = np.array([3, 5])
    trialave = 1

    # def function(self):


def gpe_ainf(V):
    ainf = 1 / (1 + np.exp(-(V + 57) / 2))
    return ainf


def gpe_hinf(V):
    hinf = 1 / (1 + np.exp((V + 58) / 12))
    return hinf


def gpe_minf(V):
    minf = 1 / (1 + np.exp(-(V + 37) / 10))
    return minf


def gpe_ninf(V):
    ninf = 1 / (1 + np.exp(-(V + 50) / 14))
    return ninf


def gpe_rinf(V):
    rinf = 1 / (1 + np.exp((V + 70) / 2))
    return rinf


def gpe_sinf(V):
    sinf = 1 / (1 + np.exp(-(V + 35) / 2))
    return sinf


def gpe_tauh(V):
    tau = 0.05 + 0.27 / (1 + np.exp((V + 40) / 12))
    return tau  # one of the two could be removed 1


def gpe_taun(V):
    tau = 0.05 + 0.27 / (1 + np.exp((V + 40) / 12))
    return tau  # one of the two could be removed 1


def th_hinf(V):
    hinf = 1 / (1 + np.exp((V + 41) / 4))
    return hinf


def th_minf(V):
    minf = 1 / (1 + np.exp(-(V + 37) / 7))
    return minf


def th_pinf(V):
    pinf = 1 / (1 + np.exp(-(V + 60) / 6.2))
    return pinf


def th_rinf(V):
    rinf = 1 / (1 + np.exp((V + 84) / 4))
    return rinf


def th_tauh(V):
    a = 0.128 * np.exp(-(V + 46) / 18)
    b = 4 / (1 + np.exp(-(V + 23) / 5))
    return 1 / (a + b)


def th_taur(V):
    tau = 0.15 * (28 + np.exp(-(V + 25) / 10.5))
    return tau


def alphah(V):
    ah = 0.128 * np.exp((-50 - V) / 18)
    return ah


def alpham(V):
    am = (0.32 * (54 + V)) / (1 - np.exp((-54 - V) / 4))
    return am


def alphan(V):
    an = (0.032 * (52 + V)) / (1 - np.exp((-52 - V) / 5))
    return an


def alphap(V):
    ap = (0.00032090 * (30 + V)) / (1 - np.exp((-30 - V) / 9))
    return ap


def betah(V):
    bh = 4 / (1 + np.exp((-27 - V) / 5))
    return bh


def betan(V):
    bn = 0.5 * np.exp((-57 - V) / 40)
    return bn


def betam(V):
    bm = 0.28 * (V + 27) / ((np.exp((27 + V) / 5)) - 1)
    return bm


def betap(V):
    bp = (-0.00032090 * (30 + V)) / (1 - np.exp((30 + V) / 9))
    return bp


def Ggaba(V):
    gb = 2 * (1 + np.tanh(V / 4))
    return gb


def stn_ainf(V):
    ainf = 1 / (1 + np.exp(-(V + 45) / 14.7))
    return ainf


def stn_binf(V):
    binf = 1 / (1 + np.exp((V + 90) / 7.5))
    return binf


def stn_cinf(V):
    cinf = 1 / (1 + np.exp(-(V + 30.6) / 5))
    return cinf


def stn_d1inf(V):
    d1inf = 1 / (1 + np.exp((V + 60) / 7.5))
    return d1inf


def stn_d2inf(V):
    d2inf = 1 / (1 + np.exp((V - 0.1) / 0.02))
    return d2inf


def stn_hinf(V):
    hinf = 1 / (1 + np.exp((V + 45.5) / 6.4))
    return hinf


def stn_minf(V):
    minf = 1 / (1 + np.exp(-(V + 40) / 8))
    return minf


def stn_ninf(V):
    ninf = 1 / (1 + np.exp(-(V + 41) / 14))
    return ninf


def stn_pinf(V):
    pinf = 1 / (1 + np.exp(-(V + 56) / 6.7))
    return pinf


def stn_qinf(V):
    qinf = 1 / (1 + np.exp((V + 85) / 5.8))
    return qinf


def stn_rinf(V):
    rinf = 1 / (1 + np.exp(-(V - 0.17) / 0.08))
    return rinf


def stn_taua(V):
    tau = 1 + 1 / (1 + np.exp(-(V + 40) / -0.5))
    return tau


def stn_taub(V):
    tau = 200 / (np.exp(-(V + 60) / -30) + np.exp(-(V + 40) / 10))
    return tau


def stn_tauc(V):
    tau = 45 + 10 / (np.exp(-(V + 27) / -20) + np.exp(-(V + 50) / 15))
    return tau


def stn_taud1(V):
    tau = 400 + 500 / (np.exp(-(V + 40) / -15) + np.exp(-(V + 20) / 20))
    return tau


def stn_tauh(V):
    tau = 24.5 / (np.exp(-(V + 50) / -15) + np.exp(-(V + 50) / 16))
    return tau


def stn_taum(V):
    tau = 0.2 + 3 / (1 + np.exp(-(V + 53) / -0.7))
    return tau


def stn_taun(V):
    tau = 11 / (np.exp(-(V + 40) / -40) + np.exp(-(V + 40) / 50))
    return tau


def stn_taup(V):
    tau = 5 + 0.33 / (np.exp(-(V + 27) / -10) + np.exp(-(V + 102) / 15))
    return tau


def stn_tauq(V):
    tau = 400 / (np.exp(-(V + 50) / -15) + np.exp(-(V + 50) / 16))
    return tau


def make_Spectrum(raw, params):
    # Compute Multitaper Spectrum
    S, f = mtspectrumpt(raw, params)

    indicies = np.array([])
    indicies = indicies.astype(int)
    for ind, val in enumerate(f):
        if val > 7 and val < 35:
            indicies = np.append(indicies, ind)

    betaf = f[indicies]
    beta = S[indicies]
    beta = np.transpose(beta)
    area = np.trapz(beta, betaf)

    return area, S, f


def mtspectrumpt(data, params):  # (*args):

    # nargin = len(args)
    nargin = 2
    nargout = 2

    if not 'data' in locals():
        print('error - Need data')

    if not 'params' in locals():
        params = np.array([])

    tapers, pad, Fs, fpass, err, trialave, params = getparams(params)

    # clear params
    data = change_row_to_column(data)

    # if (nargout > 3 and  err(1)==0):
    #  print('cannot compute error bars with err(1)=0; change params and run again')

    if nargin < 3:  # or fscorr.size == 0:
        fscorr = 0
    if nargin < 4:  # or t.size == 0:
        mintime, maxtime = minmaxsptimes(data)
        dt = 1 / Fs  # sampling time
        t = np.arange(mintime - dt, maxtime + 2 * dt, dt)  # time grid for prolates

    N = len(t)  # number of points in grid for dpss
    nextpow2_N = np.ceil(np.log2(abs(N)))
    nfft = max(2 ** (nextpow2_N + pad), N)  # number of points in fft of prolates
    nfft = int(nfft)
    f, findx = getfgrid(Fs, nfft, fpass)  # get frequency grid for evaluation
    tapers, eig = dpsschk(tapers,N,Fs)  # check tapers, it looks different than Matlabs
    [J, Msp, Nsp] = mtfftpt(data,tapers,nfft,t,f,findx)  # mt fft for point process times
    S = np.squeeze(np.mean(np.conj(J)*J,1))

    if trialave:
        S = np.squeeze(np.mean(S,1))
        Msp = np.mean(Msp)
    R = Msp*Fs
    if nargout==4:
        if fscorr==1:
            Serr=specerr(S,J,err,trialave,Nsp)
        else:
            Serr=specerr(S,J,err,trialave)

    return S, f
    # return S, f, R, Serr


def specerr(S, J, err, trialave, numsp):

    nargin = 4
    if nargin < 4:
        print('Need at least 4 input arguments')
    if err(1) == 0:
        print('Need err=[1 p] or [2 p] for error bar calculation. Make sure you are not asking for the output of Serr')

    [nf, K, C] = np.size(J)
    errchk = err[0]
    p = err[1]
    pp = 1 - p / 2
    qq = 1 - pp

    if trialave:
        dim = K * C
        C = 1
        dof = 2 * dim
        if nargin == 5:
            dof = np.fix(1 / (1 / dof + 1 / (2 * sum(numsp))))
        J = np.reshape(J, nf, dim)
    else:
        dim = K
        dof = 2 * dim * np.ones(1, C)
        for ch in range(0,C):
            if nargin == 5:
                dof[ch] = np.fix(1 / (1 / dof + 1 / (2 * numsp(ch))))

    Serr = np.zeros(2, nf, C)

    Sjk = np.zeros(dim, )
    if errchk == 1:
        Qp = chi2.ppf(pp, dof)
        Qq = chi2.ppf(qq, dof)
        Serr[0,:,:] = dof[ones(nf, 1),:] * S / Qp[np.ones(nf, 1),:]
        Serr[1,:,:] = dof[ones(nf, 1),:] * S / Qq[np.ones(nf, 1),:]
    elif errchk == 2:
        tcrit = t.ppf(pp, dim - 1)
        for k in range(0, dim):
            indices = np.setdiff(np.arange(0, dim), k)
            Jjk = J[:, indices,:] # 1 - drop projection
            eJjk = np.squeeze(np.sum(Jjk * np.conj(Jjk), 2))
            Sjk[k,:,:] = eJjk / (dim - 1) # 1 - drop spectrum
        sigma = sqrt(dim - 1) * np.squeeze(np.std(np.log(Sjk), 1, 1))
        if C == 1:
            sigma = sigma.transpose()

        conf = np.repmat(tcrit, nf, C) * sigma
        conf = np.squeeze(conf)
        Serr[0,:,:]=S * np.exp(-conf)
        Serr[1,:,:]=S * np.exp(conf)

    Serr = np.squeeze(Serr)

    return Serr


def getparams(params):

    # if not hasattr(params, 'tapers') or (params.tapers == None):  #If the tapers don't exist
    # print('tapers unspecified, defaulting to params.tapers=[3 5]')

    params.tapers = np.array([3, 5])

    if not (params == None) and (len(params.tapers) == 3):
        # Compute timebandwidth product
        TW = params.tapers[1] * params.tapers[0]
        # Compute number of tapers
        K = np.floor(2 * TW - params.tapers[2])
        params.tapers = np.array([TW, K])

    if not hasattr(params, 'pad') or (params.pad == None):
        params.pad = 0

    if not hasattr(params, 'Fs') or (params.Fs == None):
        params.Fs = 1

    if not hasattr(params, 'fpass') or (params.fpass[0] == None or params.fpass[1] == None):
        params.fpass = np.array([0, params.Fs / 2])

    if not hasattr(params, 'err') or (params.err == None):
        params.err = 0

    if not hasattr(params, 'trialave') or (params.trialave == None):
        params.trialave = 0

    tapers = params.tapers
    pad = params.pad
    Fs = params.Fs
    fpass = params.fpass
    err = params.err
    trialave = params.trialave

    return tapers, pad, Fs, fpass, err, trialave, params


def change_row_to_column(data):
    # print(data)
    # dtmp = []
    # if isstruct(data)
    #  C = len(data)
    #  if C == 1:
    #      fnames = fieldnames(data)
    #      dtmp = data.' fnames{1} ';'])
    #      data = dtmp(:)
    # else
    #  [N,C] = size(data)
    #  if N == 1 or C == 1:
    #    data=data(:)

    return data


def minmaxsptimes(data):
    dtmp = ''
    if 1:  # isstruct(data)
        ### resolve the data type issue, rather than using imported Matlab cell, create a Pythonic structure and move on
        C = data.shape[0]
        # fnames = fieldnames[data]
        mintime = np.zeros(C)
        maxtime = np.zeros(C)

        for ch in range(0, C):
            # eval(['dtmp=data(ch).' fnames{1} ';'])
            dtmp = data[ch]
            dtmpsize = dtmp.size
            if not dtmpsize == 0:
                maxtime[ch] = np.max(dtmp)
                mintime[ch] = np.min(dtmp)
            else:
                mintime[ch] = float('nan')
                maxtime[ch] = float('nan')
        maxtime = np.nanmax(maxtime)  # maximum time
        mintime = np.nanmin(mintime)  # minimum time
    else:
        dtmp = data
        if not dtmp == None:
            maxtime = np.max(dtmp)
            mintime = np.min(dtmp)
        else:
            mintime = float('nan')
            maxtime = float('nan')
    if mintime < 0:
        print('Minimum spike time is negative')

    return mintime, maxtime


def getfgrid(Fs, nfft, fpass):
    # if nargin < 3:
    #    error('Need all arguments')

    df = Fs / nfft
    f = np.arange(0, Fs + df, df)  # all possible frequencies
    f = f[0:nfft]

    if not len(fpass) == 1:
        findx_bool = np.logical_and(f >= fpass[0], f <= fpass[-1])
        findx = np.arange(0, len(findx_bool))
        findx = findx[findx_bool]
    else:
        fix = 0  # fix this
    # findx=np.min(abs(f-fpass))

    f = f[findx_bool]

    return f, findx


def mtfftpt(data, tapers, nfft, t, f, findx):


    nargin = 6
    if nargin < 6:
        print('Need all input arguments')
    if 1: #isstruct(data): # number of channels
        C = len(data)
    else:
        C = 1

    K = tapers.shape[1]  # number of tapers
    nfreq = len(f)  # number of frequencies
    if not nfreq ==len(findx):
        print('frequency information (last two arguments) inconsistent')
    if tapers.shape[0] % 2 == 1:
        tapers = tapers[0:-1, :]
        t = t[0:-1]
        #tapers = tapers[0:100000, :]

    H = fft(tapers)  # , nfft, 1 #  fft of tapers
    H = H[findx,:]  # restrict fft of tapers to required frequencies
    w = 2 * np.pi * f  # angular frequencies at which ft is to be evaluated
    Nsp = np.zeros(C,)
    Msp = np.zeros(C,)

    J = np.zeros((nfreq, K, C), dtype=np.complex_)
    for ch in range(0,C):
        if 1:  # isstruct(data):
            # fnames = fieldnames[data]
            # eval(['dtmp=data(ch).' fnames{1} ';'])
            dtmp = data[ch]
            indx_bool = np.logical_and(dtmp >= np.min(t), dtmp <= max(t))
            indx = np.arange(0, len(indx_bool))
            indx = indx[indx_bool]
            indxsize = indx.size
            if not indxsize == 0:
                dtmp=dtmp[indx]
        else:
            dtmp = data
            indx = find(dtmp >= min(t) & dtmp <= max(t))
            if not isempty(indx):
                dtmp=dtmp[indx]

        Nsp[ch] = len(dtmp)
        Msp[ch] = Nsp[ch] / len(t)
        if not abs(Msp[ch]) < 1e-6:
            data_proj = np.zeros((dtmp.shape[0], tapers.shape[1]))
            # ver 1
            for cntt in range(0, tapers.shape[1]):
                data_proj[:,cntt] =  np.interp(dtmp, t.transpose(), tapers[:, cntt])  # ver2: multiInterp2

            # better way to do it
            dtmp1 = np.zeros((1, dtmp.shape[0]))
            dtmp1[0, :] = dtmp
            w1 = np.zeros((w.shape[0], 1))
            w1[:, 0] = w
            wd = np.matmul(w1, dtmp1)

            exponential = np.exp(-1j * wd)
            J[:,:, ch]= np.matmul(exponential, data_proj) - H * Msp[ch]
        else:
            J[0:nfreq, 0:K, ch] = 0

    return J, Msp, Nsp


def dpsschk(tapers,N,Fs):

    nargin = 3
    if nargin < 3:
        print('Need all arguments')

    if tapers.shape[0]==2 and tapers.size == 2:
        tapers, eigs = dpss(N,tapers[0],tapers[1])
        tapers = tapers*sqrt(Fs)
    elif not N == tapers.shape[0]:
        print('seems to be an error in your dpss calculation; the number of time points is different from the length of the tapers')

    return tapers, eigs

def multiInterp2(x, xp, fp):
    i = np.arange(x.size)
    j = np.searchsorted(xp, x) - 1
    d = (x - xp[j]) / (xp[j + 1] - xp[j])
    return (1 - d) * fp[i, j] + fp[i, j + 1] * d

# begin

start = time.time()
# for debugging purposes only
testing_make_spectrum = 1
testing_network = 0

IT = 10
pd = 0
corstim = 0
pick_dbs_freq = 20

# !!! rng shuffle

n = 10  # number of neurons in each nucleus
tmax = 200  # ms
dt = 0.01  # ms
t = np.arange(0, tmax + dt, dt)
# Ali
ntot = int(tmax / dt) + 1  # total steps given dt

# DBS Parameters

PW = 0.3  # ms [DBS pulse width]
amplitude = 300  # nA/cm2 [DBS current amplitude]
freqs = np.arange(0, 200 + 5, 5)  # DBS frequency in Hz
pattern = freqs[pick_dbs_freq - 1]

Idbs = np.zeros(t.size)
if pick_dbs_freq != 1:
    Idbs = creatdbs()

# Create Cortical Stimulus Pulse

if corstim == 1:
    Iappco = np.zeros(t.size)
    Iappco[(1000 / dt):((1000 + 0.3) / dt)] = 350
else:
    Iappco = np.zeros(t.size)

TH_APs, STN_APs, GPe_APs, GPi_APs, Striat_APs_indr, Striat_APs_dr, Cor_APs = CTX_BG_TH_network()

if testing_make_spectrum:
    GPi_APs = np.empty((4,), dtype=object)
    GPi_APs.fill(np.array([], dtype=int))
    GPi_APs[0] = np.array(
        [])
    GPi_APs[1] = np.array(
        [0.0035, 0.013, 0.10542, 0.10546, 0.10548, 0.1055, 0.10552, 0.10778, 0.1078, 0.10782, 0.10784, 0.10786, 0.10788,
         0.1079])
    GPi_APs[2] = np.array(
        [0.0013, 0.10542, 0.10546, 0.10548, 0.1055, 0.10552, 0.10778, 0.1078, 0.10782, 0.10784, 0.10786, 0.10788, 0.1079])
    GPi_APs[3] = np.array(
        [0.0035, 0.013, 0.10542, 0.10546, 0.10548, 0.1055, 0.10552, 0.10778, 0.1078, 0.10782, 0.10784, 1.10786])

# Calculate GPi pathological low-frequency oscillatory power
params = paramsClass()

#gpi_alpha_beta_area, gpi_S, gpi_f = make_Spectrum(GPi_APs, params)

# gpi_alpha_beta_area - GPi spectral power integrated in 7-35Hz band
# gpi_S - GPi spectral power
# gpi_f - GPi spectral frequencies

#now = datetime.now()
#name = str(IT) + 'pd' + str(pattern) + now.strftime("%Y%m%d%H%M%S") + '.pkl'  # pkl instead of math
#dill.dump_session(name)

# quit

end = time.time()
print(end - start)

print("The End")

